In [14]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re


#Importation de mon module 
import Training_Doccano
from Training_Doccano import pandas_to_doccano
from Training_Doccano import train_data
from Training_Doccano import train_nlp


#Autres
import pickle
from collections import Counter
from os import path
from pathlib import Path
import glob



### 3.1.2. Application du nouveau modèle nlp sur le test_set1:

In [17]:
#####PREDICTION TEST_SET1

#importation du fichier txt de test_set1 :
with open ("./Notebook_data/Training_Doccano/train-test_sets_articles/test_set1") as f:
    test_set1_list = f.readlines()
# test_set1 = "".join(test_set1_list)
# print(test_set1_list)

#Application du nouveau nlp au test set
nlp_dir = "./Notebook_data/Training_Doccano/trained_nlp/nlp1"

nlp_test1 = spacy.load(nlp_dir)

#Modèle prédictif
Liste_pred = []
for article in test_set1_list:
    article = re.sub(r'(^ {1,2})',"",article)
    pred = nlp_test1(article)
    Liste_pred.append(pred)
    
# type pred = DOC
# Liste_pred

[Note succinte sur l’état sanitaire de nos colonies ,
 Variétés. De quelques coutumes des indigènes des Samoa et de Wallis qui tendent à disparaître ,
 Notes d'hygiène et de médecine sur l'État du Congo.  ,
 Épidémie d'ophtalmie purulente observée dans les établissements français de l'Inde, attribuée à un petit diptère du groupe des simulies ou moucherons. ,
 La peste à Tamatave ,
 Contribution à la géographie médicale. Observation d'un cas de fièvre bilieuse hémoglobinurique aux Nouvelles-Hébrides. ,
 Rapport médical sommaire sur l'épidémie de peste bubonique observée à Mongtzé (Chine) pendant les mois de juillet-août 1898.  ,
 Clinique d'Outre-Mer. Rupture du diaphragme. ,
 Notes médicales sur le Haut-Oubangui. ,
 Sur la constitution chimique des ferments solubles oxydants. ,
 Rapport sur la sérothérapie de la peste bubonique. ,
 Diagnostic bactériologique de la fièvre typhoïde dans les pays chauds. ,
 Clinique d'Outre-Mer. Trois cas de calculs vésicaux chez des enfants malgaches opé

<b>Commentaire : </b>
Le NLPsemble être assez correct dansl'ensemble!On remarque toutefois des incohérences : 
- "tsé" comme SHS
- "Terre", "malgaches" comme GPE
- "Dr Barbe" comme ORG

### 3.1.3. Evaluation du nouveau NLP

Je dois pour cela annoter moi-même le test_set1 come "gold standard" et en comparer les résultats avec ceux de la prédiction juste au dessus

In [3]:
#####GOLD TEST_SET1
path_testset1annotated="./DOCCANO/doccano-output/test_set1-annotated.jsonl"

#Conversion au format TRAIN_DATA car nécessaire pour l'évaluation 
gold_testset1=train_data(path_testset1annotated)
# gold_testset1

In [4]:
print('pred : ',len(Liste_pred),'\ngold : ',len(gold_testset1))

##Effacer la dernière entrée de Liste_pred :
# Liste_pred = Liste_pred[:-1]
print("\n\nlast element deleted\n\n")
print('pred : ',len(Liste_pred),'\ngold : ',len(gold_testset1))

# print('\n\nLast Pred : ',Liste_pred[299],'\nLast Gold : ',gold_testset1[299])

pred :  301 
gold :  300


last element deleted


pred :  301 
gold :  300


In [5]:
#Chaque EN a un : un id et un label dédié au fil des annotations.
#Ici on créé un dictionnaire de type {id_EN : label_EN}

types_labels = {0:0}

#pred.ents = Tuple contenant l'ensemble des EN reconnues
for pred in Liste_pred:
    for ent in pred.ents: 
        types_labels[ent.label] = ent.label_
    
print(types_labels)
#on obtient le résultat suivant : 
    #  384: 'GPE'
    #  4192465394259306970: 'PATH'Poulo-Condore
    #  5496441482075855828: 'SHS'
    #  385: 'LOC'
    #  11253504087253340061: 'CHEM-LABO'
    #  383: 'ORG'

{0: 0, 5496441482075855828: 'SHS', 384: 'GPE', 4192465394259306970: 'PATH', 11253504087253340061: 'CHEM-LABO', 17843529572575188375: 'ZOO-ENTO', 383: 'ORG', 385: 'LOC'}


In [7]:
# the spacy prediction is tokenised, unlike the gold standard annotation.
pred_tokens = []
pred_entities = []
pred_positions = []

# # Extraction de la prédiction EN de chaque token avec le label associé (le cas échéant) et sa posistion dans le texte

for i in range (len(Liste_pred)):
    for token in Liste_pred[i]:
        pred_tokens.append(token)
        pred_entities.append(types_labels[token.ent_type]) 
        pred_positions.append((i,token, token.idx,token.idx + len(token))) #Position de chacun des token

print(list(zip(pred_tokens,pred_entities,pred_positions)))

# on obtient en output une liste de tuples au format :
# (token,<EN>,(pos[debut],pos[fin]))

[(Note, 0, (0, Note, 0, 4)), (succinte, 0, (0, succinte, 5, 13)), (sur, 0, (0, sur, 14, 17)), (l’, 0, (0, l’, 18, 20)), (état, 0, (0, état, 20, 24)), (sanitaire, 0, (0, sanitaire, 25, 34)), (de, 0, (0, de, 35, 37)), (nos, 0, (0, nos, 38, 41)), (colonies, 0, (0, colonies, 42, 50)), (
, 0, (0, 
, 51, 52)), (Variétés, 0, (1, Variétés, 0, 8)), (., 0, (1, ., 8, 9)), (De, 0, (1, De, 10, 12)), (quelques, 0, (1, quelques, 13, 21)), (coutumes, 0, (1, coutumes, 22, 30)), (des, 0, (1, des, 31, 34)), (indigènes, 'SHS', (1, indigènes, 35, 44)), (des, 0, (1, des, 45, 48)), (Samoa, 'GPE', (1, Samoa, 49, 54)), (et, 0, (1, et, 55, 57)), (de, 0, (1, de, 58, 60)), (Wallis, 'GPE', (1, Wallis, 61, 67)), (qui, 0, (1, qui, 68, 71)), (tendent, 0, (1, tendent, 72, 79)), (à, 0, (1, à, 80, 81)), (disparaître, 0, (1, disparaître, 82, 93)), (
, 0, (1, 
, 94, 95)), (Notes, 0, (2, Notes, 0, 5)), (d', 0, (2, d', 6, 8)), (hygiène, 0, (2, hygiène, 8, 15)), (et, 0, (2, et, 16, 18)), (de, 0, (2, de, 19, 21)), (médecine, 

In [8]:
pred_positions
# gold_testset1

[(0, Note, 0, 4),
 (0, succinte, 5, 13),
 (0, sur, 14, 17),
 (0, l’, 18, 20),
 (0, état, 20, 24),
 (0, sanitaire, 25, 34),
 (0, de, 35, 37),
 (0, nos, 38, 41),
 (0, colonies, 42, 50),
 (0, , 51, 52),
 (1, Variétés, 0, 8),
 (1, ., 8, 9),
 (1, De, 10, 12),
 (1, quelques, 13, 21),
 (1, coutumes, 22, 30),
 (1, des, 31, 34),
 (1, indigènes, 35, 44),
 (1, des, 45, 48),
 (1, Samoa, 49, 54),
 (1, et, 55, 57),
 (1, de, 58, 60),
 (1, Wallis, 61, 67),
 (1, qui, 68, 71),
 (1, tendent, 72, 79),
 (1, à, 80, 81),
 (1, disparaître, 82, 93),
 (1, , 94, 95),
 (2, Notes, 0, 5),
 (2, d', 6, 8),
 (2, hygiène, 8, 15),
 (2, et, 16, 18),
 (2, de, 19, 21),
 (2, médecine, 22, 30),
 (2, sur, 31, 34),
 (2, l', 35, 37),
 (2, État, 37, 41),
 (2, du, 42, 44),
 (2, Congo, 45, 50),
 (2, ., 50, 51),
 (2,  , 52, 54),
 (3, Épidémie, 0, 8),
 (3, d', 9, 11),
 (3, ophtalmie, 11, 20),
 (3, purulente, 21, 30),
 (3, observée, 31, 39),
 (3, dans, 40, 44),
 (3, les, 45, 48),
 (3, établissements, 49, 63),
 (3, français, 64, 72),


In [9]:
# slice the gold standard text using the token positions from the prediction
gold_tokens = []  

i = 0
pos = pred_positions[i]

for pos in pred_positions:
    i = pos[0]
    if pos[0] == i:
        gold_art = gold_testset1[i]
        gold_tokens.append(gold_art["text"][pos[2]:pos[3]])

# gold_tokens

In [10]:
# create a set of all unique entity categories in the gold standard

entity_set = ['SHS', 'GPE', 'PATH', 'ZOO-ENTO', 'LOC', 'CHEM-LABO', 'ORG']


In [11]:
# this one is a bit tricky. it creates a dico from the gold standard that has all the CHARACTER positions
# that contain a given entity type (try printing if makes no sense). It basically maps the areas of the
# gold standard text where each entity is present
gold_entity_ranges = {}
entpos = []
list_ents={}
for i in entity_set:
    list_ents[i] = None
list_art_ents = {}
    
for i in range (len(gold_testset1)): #pour chaque article
    for ent in pred_positions:
#     for ent in entity_set: #opur chaque label de notre listede label 
        for entity in gold_testset1[i]['entities']: #pour chaque label annotés dans gold
            list_art_ents[str(i)]:(list(range(entity[0], entity[1])))

list_art_ents

            #if entity[2] == ent:
        

#         entpos += (list(range(entity[0], entity[1])))
#     print(gold_art,entity[2],entpos)
#     gold_entity_ranges[ent] = entpos
#     print(gold_entity_ranges)
        

# gold_entity_ranges["ORG"]
# print(gold_entity_ranges)

{}

In [13]:
# this creates a LIST for all of the TOKENS in the prediction. if the token is not in the range of
# any entity (cf last variable, gold_entity_ranges), the loop appends the label of the token, otherwise,
# it appends 0 (again, try printing)
gold_entities = []

for pos in pred_positions: #Pour l'ensemble des POS
    is_entity = False #set pardéfaut
    for label in entity_set: #pour chaque label
#         print(label)
        if set(range(pos[2], pos[3])) & set(gold_entity_ranges[label]): #Si on trouve le set de lettres dans le dico du label en question 
            is_entity = True #le pos est bien un NER
            gold_entities.append(label) #on ajoute ce label à la liste 
            break
    if not is_entity:
        gold_entities.append(0) #on met un 0 si le pos n'est pas un NER de Gold

# print(gold_entities)

KeyError: 'SHS'

In [121]:

for pos in pred_positions:
    print(pos)

(0, Note, 0, 4)
(0, succinte, 5, 13)
(0, sur, 14, 17)
(0, l’, 18, 20)
(0, état, 20, 24)
(0, sanitaire, 25, 34)
(0, de, 35, 37)
(0, nos, 38, 41)
(0, colonies, 42, 50)
(0, 
, 51, 52)
(1, Variétés, 0, 8)
(1, ., 8, 9)
(1, De, 10, 12)
(1, quelques, 13, 21)
(1, coutumes, 22, 30)
(1, des, 31, 34)
(1, indigènes, 35, 44)
(1, des, 45, 48)
(1, Samoa, 49, 54)
(1, et, 55, 57)
(1, de, 58, 60)
(1, Wallis, 61, 67)
(1, qui, 68, 71)
(1, tendent, 72, 79)
(1, à, 80, 81)
(1, disparaître, 82, 93)
(1, 
, 94, 95)
(2, Notes, 0, 5)
(2, d', 6, 8)
(2, hygiène, 8, 15)
(2, et, 16, 18)
(2, de, 19, 21)
(2, médecine, 22, 30)
(2, sur, 31, 34)
(2, l', 35, 37)
(2, État, 37, 41)
(2, du, 42, 44)
(2, Congo, 45, 50)
(2, ., 50, 51)
(2,  
, 52, 54)
(3, Épidémie, 0, 8)
(3, d', 9, 11)
(3, ophtalmie, 11, 20)
(3, purulente, 21, 30)
(3, observée, 31, 39)
(3, dans, 40, 44)
(3, les, 45, 48)
(3, établissements, 49, 63)
(3, français, 64, 72)
(3, de, 73, 75)
(3, l', 76, 78)
(3, Inde, 78, 82)
(3, ,, 82, 83)
(3, attribuée, 84, 93)
(3, à, 

(78, du, 45, 47)
(78, sommeil, 48, 55)
(78, dans, 56, 60)
(78, le, 61, 63)
(78, gouvernement, 64, 76)
(78, général, 77, 84)
(78, de, 85, 87)
(78, l', 88, 90)
(78, Afrique, 90, 97)
(78, occidentale, 98, 109)
(78, française, 110, 119)
(78, 
, 120, 121)
(79, Morbidité, 0, 9)
(79, et, 10, 12)
(79, mortalité, 13, 22)
(79, d', 23, 25)
(79, un, 25, 27)
(79, convoi, 28, 34)
(79, d', 35, 37)
(79, immigrants, 37, 47)
(79, japonais, 48, 56)
(79, en, 57, 59)
(79, Nouvelle, 60, 68)
(79, -, 68, 69)
(79, Calédonie, 69, 78)
(79, en, 79, 81)
(79, 1901, 82, 86)
(79, -, 86, 87)
(79, 1903, 87, 91)
(79, ., 91, 92)
(79, 
, 93, 94)
(80, Le, 0, 2)
(80, choléra, 3, 10)
(80, asiatique, 11, 20)
(80, à, 21, 22)
(80, Luang, 23, 28)
(80, -, 28, 29)
(80, Prabang, 29, 36)
(80, et, 37, 39)
(80, dans, 40, 44)
(80, le, 45, 47)
(80, Haut-Laos, 48, 57)
(80, en, 58, 60)
(80, 1902, 61, 65)
(80, ., 65, 66)
(80, 
, 67, 68)
(81, Des, 0, 3)
(81, différentes, 4, 15)
(81, épidémies, 16, 25)
(81, de, 26, 28)
(81, fièvre, 29, 35)
(

(139, le, 30, 32)
(139, datura, 33, 39)
(139, au, 40, 42)
(139, bagne, 43, 48)
(139, de, 49, 51)
(139, la, 52, 54)
(139, Nouvelle, 55, 63)
(139, -, 63, 64)
(139, Calédonie, 64, 73)
(139, ., 73, 74)
(139, 
, 75, 76)
(140, La, 0, 2)
(140, dengue, 3, 9)
(140, en, 10, 12)
(140, Indo-Chine, 13, 23)
(140, ., 23, 24)
(140, Épidémie, 25, 33)
(140, à, 34, 35)
(140, bord, 36, 40)
(140, de, 41, 43)
(140, La, 44, 46)
(140, Manche, 47, 53)
(140, en, 54, 56)
(140, 1907, 57, 61)
(140, ., 61, 62)
(140,  
, 63, 65)
(141, La, 0, 2)
(141, peste, 3, 8)
(141, en, 9, 11)
(141, Indo-Chine, 12, 22)
(141, (, 23, 24)
(141, historique, 24, 34)
(141, ., 34, 35)
(141, ), 35, 36)
(141, ., 36, 37)
(141, 
, 38, 39)
(142, Note, 0, 4)
(142, sur, 5, 8)
(142, l', 9, 11)
(142, azote, 11, 16)
(142, nitrique, 17, 25)
(142, de, 26, 28)
(142, quelques, 29, 37)
(142, eau, 38, 41)
(142, de, 42, 44)
(142, la, 45, 47)
(142, ville, 48, 53)
(142, de, 54, 56)
(142, Brest, 57, 62)
(142, ,, 62, 63)
(142, par, 64, 67)
(142, P., 68, 70)

(250, heures, 155, 161)
(250, ., 161, 162)
(250, -, 163, 164)
(250, Guérison, 165, 173)
(250, ., 173, 174)
(250, Par, 175, 178)
(250, les, 179, 182)
(250, docteurs, 183, 191)
(250, F., 192, 194)
(250, Michel, 195, 201)
(250, ,, 201, 202)
(250,  
, 203, 205)
(251, Variétés, 0, 8)
(251, ., 8, 9)
(251, /, 10, 11)
(251, Congrès, 12, 19)
(251, international, 20, 33)
(251, de, 34, 36)
(251, médecine, 37, 45)
(251, à, 46, 47)
(251, Lisbonne, 48, 56)
(251, en, 57, 59)
(251, 1906, 60, 64)
(251, ., 64, 65)
(251, /, 66, 67)
(251, Rangs, 68, 73)
(251, de, 74, 76)
(251, préséance, 77, 86)
(251, des, 87, 90)
(251, officiers, 91, 100)
(251, embarqués, 101, 110)
(251, ., 110, 111)
(251, Circulaire, 112, 122)
(251, ., 122, 123)
(251, [, 124, 125)
(251, C., 125, 127)
(251, Pelletan, 128, 136)
(251, ., 136, 137)
(251, ], 137, 138)
(251, 
, 139, 140)
(252, Influence, 0, 9)
(252, de, 10, 12)
(252, l', 13, 15)
(252, hygiène, 15, 22)
(252, et, 23, 25)
(252, de, 26, 28)
(252, la, 29, 31)
(252, nourriture, 32,

In [115]:

# now finally we can create two boolean lists that describe the gold standards and the predictions
# on a TOKEN LEVEL FOR ALL ENTITIESµ. for EACH LABEL, there will be a boolean list "label_gold"
# that has the length of tokens in the doc. for each token, the list has 1 if the entity in question is
# present and 0 otherwise.
# the second list 'label_pred' does the same, but this time with the prediciton. the point is to make
# the predictions comparable for each label: if the two lists, label_gold and label_pred are identical
# for a label, your model got all the entities right in that category. if label_gold has more, your model
# missed some, and vice-versa

results = {}

for label in entity_set:
    label_gold = [1 if ent==label else 0 for ent in gold_entities]
    label_pred = [1 if ent==label else 0 for ent in pred_entities]

    results[label] = (label_gold, label_pred)

In [114]:
results["GPE"]

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
